In [1]:
include("src/NursesScheduling.jl")
using .NurseSchedules
using Logging

logger = ConsoleLogger(stderr, Logging.Debug)

ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(0x0000002f) open, 0 bytes waiting))), Debug, Logging.default_metafmt, true, 0, Dict{Any,Int64}())

In [2]:
nurse_schedule = Schedule("schedules/schedule_2016_august.json")

┌ Info: Schedule loaded!
└ @ Main.NurseSchedules /home/prenc/Documents/nurse/nurse_scheduling_problem/src/Schedule.jl:13


Schedule(Dict{String,Any}("shifts" => Dict{String,Any}("nurse_2" => Any["W", "D", "R", "N", "W", "D", "W", "N", "W", "N"  …  "W", "DN", "PN", "W", "DN", "W", "DN", "N", "W", "D"],"babysitter_1" => Any["D", "N", "W", "W", "U", "U", "U", "U", "U", "U"  …  "W", "W", "N", "N", "W", "R", "DN", "W", "W", "W"],"babysitter_7" => Any["N", "W", "D", "D", "N", "W", "DN", "W", "D", "D"  …  "U", "U", "U", "U", "U", "U", "U", "U", "U", "U"],"nurse_5" => Any["W", "W", "W", "W", "N", "W", "W", "W", "W", "W"  …  "DN", "N", "R", "W", "PN", "W", "W", "W", "W", "W"],"babysitter_8" => Any["L4", "L4", "L4", "L4", "L4", "L4", "L4", "L4", "L4", "L4"  …  "L4", "L4", "L4", "L4", "L4", "L4", "L4", "L4", "L4", "L4"],"babysitter_9" => Any["N", "W", "D", "N", "N", "W", "D", "N", "W", "D"  …  "U", "U", "U", "U", "U", "U", "U", "U", "U", "U"],"nurse_1" => Any["U", "U", "U", "U", "U", "D", "W", "R", "R", "W"  …  "DN", "W", "DN", "W", "D", "D", "W", "D", "W", "DN"],"babysitter_10" => Any["W", "D", "W", "D", "D", "N", "

In [3]:
schedule_shifts = get_shifts(nurse_schedule)
workers, shifts = schedule_shifts
month_info = get_month_info(nurse_schedule)
workers_info = get_workers_info(nurse_schedule)

Dict{String,Any} with 1 entry:
  "time" => Dict{String,Any}("nurse_2"=>"FULL","babysitter_1"=>"FULL","babysitt…

In [4]:
with_logger(logger) do
    get_max_nbhd_size(nurse_schedule)
end

┌ Debug: Neighbors number from addition: 1266
└ @ Main.NurseSchedules.Neighborhood_gen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:14
┌ Debug: Neighbors number from deletion: 251
└ @ Main.NurseSchedules.Neighborhood_gen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:16
┌ Debug: Neighbors number from swap: 572
└ @ Main.NurseSchedules.Neighborhood_gen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:29


2089

In [5]:
nbhd = Neighborhood(shifts)
length(nbhd)

2089

In [6]:
with_logger(logger) do
    global schedule_score = score(schedule_shifts, month_info, workers_info)
end

┌ Debug: Worker 'nurse_2' has a disallowed shift sequence on day '27': DN -> PN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:80
┌ Debug: Worker 'nurse_2' has a disallowed shift sequence on day '32': DN -> N
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:80
┌ Debug: Worker 'nurse_2' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:97
┌ Debug: Worker 'nurse_2' does not have a long break in week: '5'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:97
┌ Debug: Worker 'babysitter_7' has a disallowed shift sequence on day '13': DN -> N
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:80
┌ Debug: Worker 'babysitter_7' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_schedul

┌ Debug: Worker 'babysitter_10' has overtime hours: '88'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'nurse_6' has overtime hours: '124'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'babysitter_5' has overtime hours: '56'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'nurse_8' has overtime hours: '28'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'nurse_4' has overtime hours: '28'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'nurse_3' has overtime hours: '120'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'babysitter_6' has overtime hours: '76'
└ @ Main.NurseSchedules.ScheduleScore

2022

In [7]:
ITERATION_NUM = 1000
best_shifts = (shifts, schedule_score)

(["W" "D" … "W" "D"; "D" "N" … "W" "W"; … ; "D" "N" … "N" "W"; "D" "N" … "DN" "W"], 2022)

In [8]:
iter_best = best_shifts
no_improvement_iters = 0

for i in 1:ITERATION_NUM
    nbhd = Neighborhood(iter_best[1])
    nghd_scores = map(shifts -> score((workers, shifts), month_info, workers_info), nbhd)
    iter_best_idx = findfirst(nghd_scores .== minimum(nghd_scores))
    iter_best = (nbhd[iter_best_idx], nghd_scores[iter_best_idx])
    
    no_improvement_iters += 1
    
    if best_shifts[2] > iter_best[2]
        println("Improved from $(best_shifts[2]) to $(iter_best[2]) in iteration '$(i)'")
        global no_improvement_iters = 0
        global best_shifts = iter_best
    end
    if no_improvement_iters > 20
        println("Local minimum! Stopping in iteration '$(i)'")
        break
    end
end

Improved from 2022 to 1968 in iteration '1'
Improved from 1968 to 1914 in iteration '2'
Improved from 1914 to 1860 in iteration '3'
Improved from 1860 to 1806 in iteration '4'
Improved from 1806 to 1752 in iteration '5'
Improved from 1752 to 1708 in iteration '6'
Improved from 1708 to 1664 in iteration '7'
Improved from 1664 to 1620 in iteration '8'
Improved from 1620 to 1576 in iteration '9'
Improved from 1576 to 1532 in iteration '10'
Improved from 1532 to 1488 in iteration '11'
Improved from 1488 to 1444 in iteration '12'
Improved from 1444 to 1410 in iteration '13'
Improved from 1410 to 1376 in iteration '14'
Improved from 1376 to 1342 in iteration '15'
Improved from 1342 to 1308 in iteration '16'
Improved from 1308 to 1276 in iteration '17'
Improved from 1276 to 1244 in iteration '18'
Improved from 1244 to 1212 in iteration '19'
Improved from 1212 to 1180 in iteration '20'
Improved from 1180 to 1148 in iteration '21'
Improved from 1148 to 1116 in iteration '22'
Improved from 1116 

In [9]:
with_logger(logger) do
    score((workers, best_shifts[1]), month_info, workers_info)
end

┌ Debug: Worker 'nurse_2' has a disallowed shift sequence on day '32': DN -> N
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:80
┌ Debug: Worker 'nurse_3' has a disallowed shift sequence on day '13': DN -> N
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:80
┌ Debug: Worker 'nurse_2' has overtime hours: '16'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'babysitter_1' has overtime hours: '12'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'babysitter_7' has overtime hours: '64'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'nurse_5' has overtime hours: '20'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:122
┌ Debug: Worker 'babysitter_8' has overtim

422